# Beating Text-Graph-17 with only Text

Current plan is following

- preprocess text into **q-a connection prediction** (question + question entities [SEP] answer + answer entities (+ linear. graph))
- finetune bert-like model (bigger=better) with some cool LoRA (this one needs to be tuned too)
- abuse augmentations for upsampling minor "correct" label examples

In [1]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch
import os
import random
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import torch.optim as optim
from sklearn.metrics import precision_score, f1_score, recall_score
from tqdm import tqdm

In [2]:
SEED = 42

torch.manual_seed(SEED)
torch.random.manual_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.random.manual_seed(SEED)
torch.cuda.random.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Data Preproc

In [3]:
train_path = 'data/tsv/train.tsv'
test_path = 'data/tsv/test.tsv'

from data.dataset import TextGraphDataset

## Model prep and finetuning

In [4]:
# Load model directly
# model_name = "whaleloops/phrase-bert"
model_name = "sentence-transformers/all-MiniLM-L6-v2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
pretrained_bert = AutoModel.from_pretrained(model_name)

In [5]:
class QuestionClassifier(nn.Module):
    def __init__(self, pretrained_bert):
        super().__init__()
        self.bert_backbone = pretrained_bert
        self.hidden_size = pretrained_bert.config.hidden_size
        self.head = nn.Sequential(
            nn.Linear(self.hidden_size, self.hidden_size // 2),
            nn.ELU(),
            nn.Linear(self.hidden_size // 2, 1)
        )

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        outputs = self.bert_backbone(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        last_hidden_state = outputs.last_hidden_state  # Access the last hidden states
        pooled_output = last_hidden_state[:, 0, :]  # Take the [CLS] token representation
        logits = self.head(pooled_output)
        return logits
    
model = QuestionClassifier(
    pretrained_bert
).to(DEVICE)

for p in model.bert_backbone.parameters():
    p.requires_grad = False

In [6]:
# %pip install peft -q

In [7]:
from peft import LoraConfig, LoraModel

LORA_RANK=16
LORA_ALPHA=32.
LORA_DROPOUT=1e-1

config = LoraConfig(
    task_type="SEQ_CLS",
    r=LORA_RANK,
    lora_alpha=LORA_ALPHA,
    target_modules=["query", "value"],
    lora_dropout=LORA_DROPOUT,
    use_rslora=True,
)

lora_model = LoraModel(model, config, "default")

for p in lora_model.head.parameters():
    p.requires_grad = True

In [8]:
def get_trainable_params(model: nn.Module):
    params = []
    for name, p in model.named_parameters():
        if p.requires_grad:
            params.append(p)
    return params

trainable_params = get_trainable_params(lora_model)
len(trainable_params)

28

In [9]:
def train_epoch(model, loader, optimizer, loss_fn):
    model.train()

    avg_loss = 0.

    predictions = []
    true_labels = []
    
    for i, batch in tqdm(enumerate(loader), total=len(loader)):

        optimizer.zero_grad()

        input_ids = batch["input_ids"].to(DEVICE)
        token_type_ids = batch["token_type_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE).float()
        logits = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze()
        loss = loss_fn(logits, labels)
        loss.backward()
        optimizer.step()

        avg_loss += loss.item()
        with torch.no_grad():
            preds = F.sigmoid(logits).detach().cpu().numpy()
            preds = (preds > 0.5) * 1
            y_true = labels.detach().cpu().numpy()
            
            predictions += preds.tolist()
            true_labels += y_true.tolist()
    
    avg_loss /= len(loader) + 1
    f1 = f1_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)    
    
    return avg_loss, f1, precision, recall


@torch.no_grad()
def eval_epoch(model, loader, loss_fn):
    model.eval()

    avg_loss = 0.
    predictions, true_labels = [], []

    for i, batch in tqdm(enumerate(loader), total=len(loader)):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        token_type_ids = batch["token_type_ids"].to(DEVICE)
        labels = batch["labels"].to(DEVICE).float()
        
        logits = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze()
        loss = loss_fn(logits, labels)
        
        avg_loss += loss.item()
        preds = F.sigmoid(logits).detach().cpu().numpy()
        preds = (preds > 0.5) * 1
        y_true = labels.detach().cpu().numpy()
        predictions += preds.tolist()
        true_labels += y_true.tolist()

    avg_loss /= len(loader)
    f1 = f1_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)

    return avg_loss, f1, precision, recall


def train(model, train_loader, val_loader, optimizer, loss_fn, epochs=10):
    for e in range(epochs):
        loss, f1, prec, rec = train_epoch(model, train_loader, optimizer, loss_fn)
        print(f"Train epoch {e + 1} - loss: {loss:.3f}, f1: {f1:.3f}, precision: {prec:.3f}, recall: {rec:.3f}")
        
        loss, f1, prec, rec = eval_epoch(model, val_loader, loss_fn)
        print(f"Eval epoch {e + 1} - loss: {loss:.3f}, f1: {f1:.3f}, precision: {prec:.3f}, recall: {rec:.3f}")
        

## Training, evaluation and submit

In [10]:
BATCH_SIZE=64
MAX_LENGTH=256
EPOCHS=50

INCLUDE_GRAPH = True

from sklearn.utils.class_weight import compute_sample_weight
from torch.utils.data import WeightedRandomSampler

train_ds = TextGraphDataset(tokenizer, MAX_LENGTH, train_path=train_path, test_path=test_path, 
                            split='train', include_graph=INCLUDE_GRAPH)
dev_ds = TextGraphDataset(tokenizer, MAX_LENGTH, train_path=train_path, test_path=test_path,
                          split='val', include_graph=INCLUDE_GRAPH)
test_ds = TextGraphDataset(tokenizer, MAX_LENGTH, train_path=train_path, test_path=test_path,
                           split='test', include_graph=INCLUDE_GRAPH)

weights = compute_sample_weight('balanced', train_ds.labels)
sampler = WeightedRandomSampler(weights, len(weights)) # we will oversample correct answers :)

CONFIG_DATALOADER = {"num_workers":4, "pin_memory":True}
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, **CONFIG_DATALOADER)
#train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
dev_loader = DataLoader(dev_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, **CONFIG_DATALOADER)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False, **CONFIG_DATALOADER)

C:\keep_pc\Skoltech-PC\Courses\Term 4\DL for NLP\Task 1\work-2-from_repo\text-graph\data\dataset.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["graph"] = self.df["graph"].apply(eval)
C:\keep_pc\Skoltech-PC\Courses\Term 4\DL for NLP\Task 1\work-2-from_repo\text-graph\data\dataset.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["graph"] = self.df["graph"].apply(eval)
C:\keep_pc\Skoltech-PC\Courses\Term 4\DL for NLP\Task 1\work-2-from_repo\text-graph\data\dataset.py:88: SettingWithC

In [11]:
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(params=trainable_params, lr=3e-4)

In [12]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

### PQlet run - with linearized graph

In [13]:
%%time 

train(
    lora_model,
    train_loader,
    dev_loader,
    optimizer,
    loss_fn,
    epochs=EPOCHS
)

# torch.save(model.state_dict(), "phrase_bert-lora-fixed_oversampling-pqlet.pth")
torch.save(model.state_dict(), "all-MiniLM-L6-v2--lora-fixed_oversampling-pqlet-includegraphs.pth")

100%|████████████████████████████████████████████████████████████████████████████████| 424/424 [01:52<00:00,  3.77it/s]


Train epoch 1 - loss: 0.349, f1: 0.823, precision: 0.785, recall: 0.864


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:05<00:00,  8.03it/s]


Eval epoch 1 - loss: 0.280, f1: 0.505, precision: 0.356, recall: 0.869


100%|████████████████████████████████████████████████████████████████████████████████| 424/424 [01:51<00:00,  3.80it/s]


...

Eval epoch 49 - loss: 0.123, f1: 0.826, precision: 0.760, recall: 0.906


100%|████████████████████████████████████████████████████████████████████████████████| 424/424 [01:51<00:00,  3.82it/s]


Train epoch 50 - loss: 0.062, f1: 0.979, precision: 0.972, recall: 0.986


100%|██████████████████████████████████████████████████████████████████████████████████| 47/47 [00:05<00:00,  8.07it/s]

Eval epoch 50 - loss: 0.148, f1: 0.806, precision: 0.728, recall: 0.902
CPU times: total: 1h 41min 1s
Wall time: 1h 52min 49s


In [38]:
train_graph_linear = train_loader.dataset.df['graph'].apply(eval).apply(lambda x: linearize_graph(x, '[SEP]'))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [51]:
train_loader.dataset.df['graph'][5]

"{'nodes': [{'type': 'QUESTIONS_ENTITY', 'name_': 'Q794', 'id': 0, 'label': 'Iran'}, {'type': 'ANSWER_CANDIDATE_ENTITY', 'name_': 'Q34448', 'id': 1, 'label': 'Mahmoud Ahmadinejad'}], 'links': [{'name_': 'P17', 'source': 0, 'target': 0, 'label': 'country'}, {'name_': 'P6', 'source': 0, 'target': 1, 'label': 'head of government'}, {'name_': 'P27', 'source': 1, 'target': 0, 'label': 'country of citizenship'}]}"

In [52]:
linearize_graph(train_loader.dataset.df['graph'].apply(eval)[5], '[SEP]')

[{'type': 'QUESTIONS_ENTITY', 'name_': 'Q794', 'id': 0, 'label': 'Iran'}, {'type': 'ANSWER_CANDIDATE_ENTITY', 'name_': 'Q34448', 'id': 1, 'label': 'Mahmoud Ahmadinejad'}]


' Iran, country, Iran ; Iran, head of government, [SEP] Mahmoud Ahmadinejad [SEP] ; [SEP] Mahmoud Ahmadinejad [SEP], country of citizenship, Iran ;'

Decoding input examples

In [61]:
# INCLUDE_GRAPHS = False
tokenizer.decode(train_loader.dataset[5]['input_ids'], skip_special_tokens=False)

'[CLS] iran : whst is the name of the head of state and highest ranking political and religious authority in iran? [SEP] mahmoud ahmadinejad [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [59]:
# INCLUDE_GRAPHS = True
tokenizer.decode(train_loader.dataset[5]['input_ids'], skip_special_tokens=False)

'[CLS] iran : whst is the name of the head of state and highest ranking political and religious authority in iran? [SEP] iran, country, iran ; iran, head of government, [SEP] mahmoud ahmadinejad [SEP] ; [SEP] mahmoud ahmadinejad [SEP], country of citizenship, iran ; [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

### Original run shredder67

In [41]:
# train(
#     lora_model,
#     train_loader,
#     dev_loader,
#     optimizer,
#     loss_fn,
#     epochs=EPOCHS
# )

Train epoch 1 - loss: 0.636, f1: 0.637, precision: 0.618, recall: 0.657
Eval epoch 1 - loss: 0.565, f1: 0.286, precision: 0.180, recall: 0.696
Train epoch 2 - loss: 0.550, f1: 0.732, precision: 0.687, recall: 0.784
Eval epoch 2 - loss: 0.757, f1: 0.285, precision: 0.171, recall: 0.870
Train epoch 3 - loss: 0.509, f1: 0.766, precision: 0.717, recall: 0.822
Eval epoch 3 - loss: 0.533, f1: 0.340, precision: 0.215, recall: 0.813
Train epoch 4 - loss: 0.473, f1: 0.791, precision: 0.738, recall: 0.853
Eval epoch 4 - loss: 0.523, f1: 0.352, precision: 0.221, recall: 0.873
Train epoch 5 - loss: 0.459, f1: 0.800, precision: 0.751, recall: 0.856
Eval epoch 5 - loss: 0.438, f1: 0.404, precision: 0.269, recall: 0.809
Train epoch 6 - loss: 0.416, f1: 0.820, precision: 0.778, recall: 0.867
Eval epoch 6 - loss: 0.422, f1: 0.434, precision: 0.299, recall: 0.786
Train epoch 7 - loss: 0.390, f1: 0.837, precision: 0.794, recall: 0.885
Eval epoch 7 - loss: 0.415, f1: 0.432, precision: 0.296, recall: 0.803

In [42]:
# _, f1, prec, rec = eval_epoch(model, test_loader, loss_fn)
# print(f"Performance on hold-out test - f1: {f1:.2f}, precision: {prec:.2f}, recall: {rec:.2f}")

Performance on hold-out test - f1: 0.53, precision: 0.39, recall: 0.82


In [43]:
# torch.save(model.state_dict(), "phrase_bert_lora_fixed_oversampling.pth")

In [ ]:
# load saved model (maybe there is a better way with PEFT around, but...)

In [44]:
# eval_ds = TextGraphDataset(tokenizer, max_length=MAX_LENGTH, split='eval')
# eval_df = eval_ds.df

In [48]:
# q = eval_df.loc[0, "question"]
# ids = eval_df.index[eval_df['question'] == q].tolist()
# for id in ids:
#     print(eval_ds.questions[id])

After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?
After publishing A Time to Kill, which book did its author begin working on immediately?


In [22]:
@torch.no_grad()
def make_submit_predictions(model, tokenizer, include_graph, filename='test_result_1.tsv'):
    model.eval()
    eval_ds = TextGraphDataset(tokenizer, max_length=MAX_LENGTH,  train_path=train_path, test_path=test_path,
                               split='eval', include_graph=include_graph)
    preds = []
    for idx, data in tqdm(enumerate(eval_ds)):
        input_ids = data["input_ids"].to(DEVICE).unsqueeze(0)
        attention_mask = data["attention_mask"].to(DEVICE).unsqueeze(0)
        token_type_ids = data["token_type_ids"].to(DEVICE).unsqueeze(0)
        
        logit = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze()
        pred = (logit.detach().cpu().numpy() > 0) * 1
        preds.append(pred)

    df = eval_ds.df
    df['prediction'] = preds
    df['prediction'] = df['prediction'].astype(int)
    df[["sample_id", "prediction"]].to_csv(filename, sep='\t', index=False)

@torch.no_grad()
def make_submit_predictions_ranked(model, tokenizer, include_graph, filename='test_result_2.tsv'):
    """based of Vika's idea - select all candidate answers for questions, select one with max prob"""
    model.eval()
    eval_ds = TextGraphDataset(tokenizer, max_length=MAX_LENGTH,  train_path=train_path, test_path=test_path,
                               split='eval', include_graph=include_graph)
    eval_df = eval_ds.df
    eval_df["correct"] = False

    for question in tqdm(eval_df['question'].unique()):
        ids = eval_df.index[eval_df['question'] == question].tolist()
        
        logits = []
        for idx in ids:
            data = eval_ds[idx]
            input_ids = data["input_ids"].to(DEVICE).unsqueeze(0)
            attention_mask = data["attention_mask"].to(DEVICE).unsqueeze(0)
            token_type_ids = data["token_type_ids"].to(DEVICE).unsqueeze(0)
            
            logit = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids).squeeze()
            logits.append(logit.detach().cpu().item())

        right_ans_id = ids[np.argmax(logits)]
        eval_df.loc[right_ans_id, 'correct'] = True

    eval_df['prediction'] = eval_df['correct']
    eval_df['prediction'] = eval_df['prediction'].astype(int)
    eval_df[["sample_id", "prediction"]].to_csv(filename, sep='\t', index=False)

In [132]:
# make_submit_predictions(
#     model,
#     tokenizer
# )

In [54]:
# make_submit_predictions_ranked(
#     model,
#     tokenizer
# )

### Complete retrain on full data (for best result)

In [24]:
from sklearn.utils.class_weight import compute_sample_weight
from torch.utils.data import WeightedRandomSampler

full_ds = TextGraphDataset(tokenizer, MAX_LENGTH, train_path=train_path, test_path=test_path,
                           split='full', include_graph=INCLUDE_GRAPH)
weights = compute_sample_weight('balanced', full_ds.labels)
sampler = WeightedRandomSampler(weights, len(weights)) # we will oversample correct answers :)
loader = DataLoader(full_ds, batch_size=BATCH_SIZE, sampler=sampler, **CONFIG_DATALOADER)

In [26]:
pretrained_bert = AutoModel.from_pretrained(model_name)
model = QuestionClassifier(
    pretrained_bert
).to(DEVICE)

for p in model.bert_backbone.parameters():
    p.requires_grad = False
lora_model = LoraModel(model, config, "default")

for p in lora_model.head.parameters():
    p.requires_grad = True
trainable_params = get_trainable_params(lora_model)
len(trainable_params)

optimizer = optim.AdamW(params=trainable_params, lr=3e-4)

In [ ]:
for e in range(EPOCHS):
    loss, f1, prec, rec = train_epoch(model, loader, optimizer, loss_fn)
    print(f"Train epoch {e} - loss: {loss:.3f}, f1: {f1:.3f}, precision: {prec:.3f}, recall: {rec:.3f}")
    
torch.save(model.state_dict(), "all-MiniLM-L6-v2--lora-fixed_oversampling-pqlet-includegraphs-50efull-last.pth")

100%|████████████████████████████████████████████████████████████████████████████████| 589/589 [02:41<00:00,  3.65it/s]


Train epoch 0 - loss: 0.346, f1: 0.830, precision: 0.788, recall: 0.876


100%|████████████████████████████████████████████████████████████████████████████████| 589/589 [02:55<00:00,  3.36it/s]


Train epoch 1 - loss: 0.276, f1: 0.869, precision: 0.831, recall: 0.910


100%|████████████████████████████████████████████████████████████████████████████████| 589/589 [02:43<00:00,  3.61it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████| 589/589 [02:42<00:00,  3.62it/s]


Train epoch 4 - loss: 0.209, f1: 0.910, precision: 0.875, recall: 0.949


100%|████████████████████████████████████████████████████████████████████████████████| 589/589 [02:42<00:00,  3.62it/s]


Train epoch 5 - loss: 0.190, f1: 0.921, precision: 0.888, recall: 0.956


  3%|██▌                                                                              | 19/589 [00:05<02:34,  3.69it/s]

In [ ]:
make_submit_predictions_ranked(
    model,
    tokenizer,
    include_graph=INCLUDE_GRAPH,
    filename="test_res_overfit_pqlet_includegraphs.csv",
)